In [0]:
# Mixup model on the cifar10 dataset
#We have refered to the code from below github link
#https://github.com/yu4u/mixup-generator
# This link contains the mixup generator code we used 
#to augment data using mixup_generator and we have used it for our analysis
#Below is the licence copyright for mixup generator imported in this code
#MIT License
#Copyright (c) 2017 Yusuke Uchida

#@author : Prashant Jadhav

import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from mixup_generator import MixupGenerator #This particular loc is used from github link https://github.com/yu4u/mixup-generator usage section
import seaborn as sns

# load train and test dataset
def load_dataset():
	# load dataset
	(train_X, train_Y), (test_X, test_Y1) = cifar10.load_data()
	# one hot encode target values
	train_Y = to_categorical(train_Y)
	test_Y = to_categorical(test_Y1)
	return train_X, train_Y, test_X, test_Y, test_Y1

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	training_norm = train.astype('float32')
	testing_norm = test.astype('float32')
	# normalize to range 0-1
	training_norm = training_norm / 255.0
	testing_norm = testing_norm / 255.0
	# return normalized images
	return training_norm, testing_norm

# define cnn modez
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001), input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', kernel_regularizer=l2(0.001)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.001)))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	
	return model

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['acc'], color='blue', label='train')
	pyplot.plot(history.history['val_acc'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	train_X, train_Y, test_X, test_Y, test_Y1 = load_dataset()
	# prepare pixel data
	train_X, test_X = prep_pixels(train_X, test_X)
	# define model
	model = define_model()
	# fit model
	#history = model.fit(train_X, train_Y, epochs=100, batch_size=500, validation_data=(test_X, test_Y), verbose=1)
	datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)
	training_generator = MixupGenerator(train_X, train_Y, batch_size=500, alpha=0.2, datagen=datagen)()
	history=model.fit_generator(generator=training_generator,
                        steps_per_epoch=500 ,
                        validation_data=(test_X, test_Y),
                        epochs=50, verbose=1)
	#confusion matrix
	y_pred_ohe = model.predict(test_X) 
	y_pred_labels = np.argmax(y_pred_ohe, axis=1) 
	confusion_matrix = metrics.confusion_matrix(y_true=test_Y1, y_pred=y_pred_labels)
	figure = pyplot.figure(figsize=(8, 8))
	sns.heatmap(confusion_matrix, annot=False,cmap=pyplot.cm.Blues)
	pyplot.tight_layout()
	pyplot.ylabel('True label')
	pyplot.xlabel('Predicted label')
	pyplot.imshow(confusion_matrix)
	# evaluate model
	_, acc = model.evaluate(test_X, test_Y, verbose=0)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	#summarize_diagnostics(history)

# entry point, run the test harness
run_test_harness()

Using TensorFlow backend.


170500096/170498071 [==============================] - 11s 0us/step






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/50





500/500 [==============================] - 165s 331ms/step - loss: 3.0678 - acc: 0.3268 - val_loss: 2.6855 - val_acc: 0.4448
Epoch 2/50
500/500 [==============================] - 157s 313ms/step - loss: 2.7882 - acc: 0.4374 - val_loss: 2.5043 - val_acc: 0.5049
Epoch 3/50
500/500 [==============================] - 156s 312ms/step - loss: 2.6854 - acc: 0.4753 - val_loss: 2.3907 - val_acc: 0.5419
Epoch 4/50
500/500 [==============================] - 157s 313ms/step - loss: 2.5983 - acc: 0.5064 - val_loss: 2.2887 - val_acc: 0.5779
Epoch 5/50
500/500 [==============================] - 157s 313ms/step - loss: 2.5276 - acc: 0.5306 - val_loss: 2.2700 - val_acc: 0.5777
Epoch 6/50
500/500 [==============================] - 157s 314ms/step - loss: 2.4672 - acc: 0.5489 - val_loss: 2.1967 - val_acc: 0.5945
Epoch 7/